In [204]:
import numpy as np
import math

# MVP

$$$
\begin{bmatrix}
x_{sceen} \\
y_{sceen} \\
z_{sceen} \\
1
\end{bmatrix} = MVP * \begin{bmatrix}
x_{model} \\
y_{model} \\
z_{model} \\
1
\end{bmatrix}
$$$


## Model

$$$
\begin{bmatrix}
x_{model} \\
y_{model} \\
z_{model} \\
1
\end{bmatrix} = M * \begin{bmatrix}
x_{local} \\
y_{local} \\
z_{local} \\
1
\end{bmatrix}
$$$

In [205]:
model = np.array([
        [0, 2, 0, 2],
        [1, 0, 0, 3],
        [0, 0, 3, 1],
        [0, 0, 0, 1],
    ])


## View

$$$
\begin{bmatrix}
x_{view} \\
y_{view} \\
z_{view} \\
1
\end{bmatrix} = V * \begin{bmatrix}
x_{model} \\
y_{model} \\
z_{model} \\
1
\end{bmatrix}
$$$


## Projection

### Perspective

$1 > Z_{screen} > -1$

$f > Z_{view} > n$

$$$
\begin{bmatrix}
x_{projection} \\
y_{projection} \\
z_{projection} \\
w_{projection}
\end{bmatrix} = \begin{bmatrix}
\frac{n}{r} & 0 & 0 & 0 \\
0 & \frac{n}{t} & 0 & 0 \\
0 & 0 & \frac{-(f+n)}{f-n} & \frac{-2fn}{f-n} \\
0 & 0 & -1 & 0  
\end{bmatrix}* \begin{bmatrix}
x_{view} \\
y_{view} \\
z_{view} \\
1
\end{bmatrix}
$$$

$$$
\begin{bmatrix}
x_{screen} \\
y_{screen} \\
z_{screen} \\
1
\end{bmatrix} = \begin{bmatrix}
\frac{x_{projection}}{w_{projection}} \\
\frac{y_{projection}}{w_{projection}} \\
\frac{z_{projection}}{w_{projection}} \\
\frac{w_{projection}}{w_{projection}}
\end{bmatrix}
$$$

In [206]:
n = 2
f = 10
width = 4
height = 2
r = width / 2
t = height / 2
pointNear = np.array([r, t, -n, 1])
pointFar = np.array([r * f/n, t * f/n, -f, 1])
projection = np.array([
    [n/r, 0, 0, 0],
    [0, n/t, 0, 0],
    [0, 0, -(f+n)/(f-n), -2*f*n/(f-n)],
    [0, 0, -1, 0]
])
print(projection)
tmp = np.matmul(projection, pointNear)
print(tmp / tmp[3])
tmp = np.matmul(projection, pointFar)
print(tmp / tmp[3])

[[ 1.   0.   0.   0. ]
 [ 0.   2.   0.   0. ]
 [ 0.   0.  -1.5 -5. ]
 [ 0.   0.  -1.   0. ]]
[ 1.  1. -1.  1.]
[1. 1. 1. 1.]


### Orthographic

$$$
\begin{bmatrix}
x_{screen} \\
y_{screen} \\
z_{screen} \\
1
\end{bmatrix} = \begin{bmatrix}
\frac{1}{r} & 0 & 0 & 0 \\
0 & \frac{1}{t} & 0 & 0 \\
0 & 0 & \frac{-2}{f-n} & -\frac{f+n}{f-n} \\
0 & 0 & 0 & 1  
\end{bmatrix} * \begin{bmatrix}
x_{view} \\
y_{view} \\
z_{view} \\
1
\end{bmatrix}
$$$


## Inverse

$$$
\begin{bmatrix}
x_{worl} * w \\
y_{worl} * w \\
z_{worl} * w \\
w
\end{bmatrix} = MVP^{-1} * \begin{bmatrix}
x_{screen} \\
y_{screen} \\
z_{screen} \\
1
\end{bmatrix}
$$$

In [207]:
MVP = np.matmul(projection, model)
MVP

array([[ 0. ,  2. ,  0. ,  2. ],
       [ 2. ,  0. ,  0. ,  6. ],
       [ 0. ,  0. , -4.5, -6.5],
       [ 0. ,  0. , -3. , -1. ]])

In [208]:
wordPoint = np.array([1, 2, -3, 1])
tmp = np.matmul(MVP, wordPoint)
screenPoint = tmp / tmp[3]
screenPoint

array([0.75 , 1.   , 0.875, 1.   ])

In [209]:
MVPinv = np.linalg.inv(MVP)
MVPinv

array([[ 0.        ,  0.5       ,  0.6       , -0.9       ],
       [ 0.5       ,  0.        ,  0.2       , -0.3       ],
       [ 0.        ,  0.        ,  0.06666667, -0.43333333],
       [ 0.        ,  0.        , -0.2       ,  0.3       ]])

In [210]:
tmp = np.matmul(MVPinv, screenPoint)
tmp / tmp[3]

array([ 1.,  2., -3.,  1.])

## Partial Inverse

$$$
\begin{bmatrix}
x_{worl} * w \\
y_{worl} * w \\
z_{worl} * w \\
w
\end{bmatrix} = \begin{bmatrix}
m11 & m12 & m13 & m14 \\
m21 & m22 & m23 & m24 \\
m31 & m32 & m33 & m34 \\
m41 & m42 & m43 & m44
\end{bmatrix} * \begin{bmatrix}
x_{screen} \\
y_{screen} \\
z_{screen} \\
1
\end{bmatrix}
$$$

$$$
\begin{bmatrix}
x_{worl} * w \\
y_{worl} * w \\
z_{worl} * w \\
w
\end{bmatrix} = \begin{bmatrix}
m11 * x_{screen} + m12 * y_{screen} + m13 * z_{screen} + m14 \\
m21 * x_{screen} + m22 * y_{screen} + m23 * z_{screen} + m24 \\
m31 * x_{screen} + m32 * y_{screen} + m33 * z_{screen} + m34 \\
m41 * x_{screen} + m42 * y_{screen} + m43 * z_{screen} + m44 
\end{bmatrix}
$$$

$$$
\begin{bmatrix}
x_{worl} \\
y_{worl} \\
z_{worl} \\
1
\end{bmatrix} = \begin{bmatrix}
\frac{m11 * x_{screen} + m12 * y_{screen} + m13 * z_{screen} + m14}{m41 * x_{screen} + m42 * y_{screen} + m43 * z_{screen} + m44} \\
\frac{m21 * x_{screen} + m22 * y_{screen} + m23 * z_{screen} + m24}{m41 * x_{screen} + m42 * y_{screen} + m43 * z_{screen} + m44} \\
\frac{m31 * x_{screen} + m32 * y_{screen} + m33 * z_{screen} + m34}{m41 * x_{screen} + m42 * y_{screen} + m43 * z_{screen} + m44} \\
1
\end{bmatrix}
$$$

$$$
z_{worl} = \frac{m31 * x_{screen} + m32 * y_{screen} + m33 * z_{screen} + m34}{m41 * x_{screen} + m42 * y_{screen} + m43 * z_{screen} + m44} \\

(m41 * x_{screen} + m42 * y_{screen} + m43 * z_{screen} + m44) * z_{worl} = m31 * x_{screen} + m32 * y_{screen} + m33 * z_{screen} + m34 \\

m41 * x_{screen} * z_{worl} + m42 * y_{screen} * z_{worl} + m43 * z_{screen} * z_{worl} + m44 * z_{worl} = m31 * x_{screen} + m32 * y_{screen} + m33 * z_{screen} + m34 \\

x_{screen} * (m41 * z_{worl} - m31) + y_{screen} * (m42 * z_{worl} - m32) + z_{screen} * (m43 * z_{worl} - m33) + m44 * z_{worl} = m34 \\

z_{screen} = \frac{m34 - x_{screen} * (m41 * z_{worl} - m31) - y_{screen} * (m42 * z_{worl} - m32) - m44 * z_{worl}}{m43 * z_{worl} - m33}
$$$

In [211]:
MVP = np.array(
    [[-1.81,     0,    -4,     0],
     [    0, -2.37, -4.36, -6.14],
     [    0, 0.445, 14.7, -32.8],
     [    0,     0,    -1,     0]])
wordPoint = np.array([0, 0, 1, 1])
screenPoint = np.matmul(MVP, wordPoint)
screenPoint = screenPoint / screenPoint[3]

m = np.linalg.inv(MVP)

xScreen = 0.46
yScreen = 0.67
zWorld = -20
print(wordPoint)
print(screenPoint)
print(m)
(m[2][3] - xScreen * (m[3][0] * zWorld - m[2][0]) - yScreen * (m[3][1] * zWorld - m[2][1]) - zWorld * m[3][3]) / (zWorld * m[3][2] - m[2][2])

[0 0 1 1]
[ 4.  10.5 18.1  1. ]
[[-5.52486188e-01  0.00000000e+00 -1.76750209e-17  2.20994475e+00]
 [ 0.00000000e+00 -4.07613930e-01  7.63033393e-02  2.89885582e+00]
 [ 0.00000000e+00  0.00000000e+00  7.99794694e-18 -1.00000000e+00]
 [ 0.00000000e+00 -5.53012801e-03 -2.94525919e-02 -4.08841743e-01]]


-15.70479535864979

In [212]:
tmp = np.matmul(MVPinv, screenPoint)
tmp / tmp[3]

array([-4.5813253 , -1.60240964, -0.23293173,  1.        ])

In [213]:
tmp = np.array([
    m[0][0] * screenPoint[0] + m[0][1] * screenPoint[1] + m[0][2] * screenPoint[2] + m[0][3],
    m[1][0] * screenPoint[0] + m[1][1] * screenPoint[1] + m[1][2] * screenPoint[2] + m[1][3],
    m[2][0] * screenPoint[0] + m[2][1] * screenPoint[1] + m[2][2] * screenPoint[2] + m[2][3],
    m[3][0] * screenPoint[0] + m[3][1] * screenPoint[1] + m[3][2] * screenPoint[2] + m[3][3]
])
tmp / tmp[3]

array([8.8817842e-16, 4.4408921e-16, 1.0000000e+00, 1.0000000e+00])

In [214]:
np.array([
    (m[0][0] * screenPoint[0] + m[0][1] * screenPoint[1] + m[0][2] * screenPoint[2] + m[0][3]) /
    (m[3][0] * screenPoint[0] + m[3][1] * screenPoint[1] + m[3][2] * screenPoint[2] + m[3][3]),
    (m[1][0] * screenPoint[0] + m[1][1] * screenPoint[1] + m[1][2] * screenPoint[2] + m[1][3]) /
    (m[3][0] * screenPoint[0] + m[3][1] * screenPoint[1] + m[3][2] * screenPoint[2] + m[3][3]),
    (m[2][0] * screenPoint[0] + m[2][1] * screenPoint[1] + m[2][2] * screenPoint[2] + m[2][3]) /
    (m[3][0] * screenPoint[0] + m[3][1] * screenPoint[1] + m[3][2] * screenPoint[2] + m[3][3]),
    1
])

array([8.8817842e-16, 4.4408921e-16, 1.0000000e+00, 1.0000000e+00])

In [215]:
print(wordPoint[2])

(m[2][0] * screenPoint[0] + m[2][1] * screenPoint[1] + m[2][2] * screenPoint[2] + m[2][3]) / (m[3][0] * screenPoint[0] + m[3][1] * screenPoint[1] + m[3][2] * screenPoint[2] + m[3][3])

1


0.9999999999999999

In [216]:
print(wordPoint[2] * m[3][0] * screenPoint[0] + wordPoint[2] * m[3][1] * screenPoint[1] + wordPoint[2] * m[3][2] * screenPoint[2] + wordPoint[2] * m[3][3])

(m[2][0] * screenPoint[0] + m[2][1] * screenPoint[1] + m[2][2] * screenPoint[2] + m[2][3])

-0.9999999999999999


-0.9999999999999998

In [217]:
print(wordPoint[2] * m[3][2] * screenPoint[2] - m[2][2] * screenPoint[2])

(m[2][0] * screenPoint[0] + m[2][1] * screenPoint[1] + m[2][3] - wordPoint[2] * m[3][0] * screenPoint[0] - wordPoint[2] * m[3][1] * screenPoint[1] - wordPoint[2] * m[3][3])

-0.5330919132130293


-0.5330919132130292

In [218]:
print(screenPoint[2])

(m[2][0] * screenPoint[0] + m[2][1] * screenPoint[1] + m[2][3] - wordPoint[2] * m[3][0] * screenPoint[0] - wordPoint[2] * m[3][1] * screenPoint[1] - wordPoint[2] * m[3][3]) / (wordPoint[2] * m[3][2] - m[2][2])

18.099999999999998


18.099999999999994

In [219]:
print(screenPoint[2])

(m[2][3] + xScreen * (m[2][0] - zWorld * m[3][0]) + yScreen * (m[2][1] - zWorld * m[3][1]) - zWorld * m[3][3]) / (zWorld * m[3][2] - m[2][2])

18.099999999999998


-15.70479535864979